In [1]:
import os
#form data types in file. SeqIO handles sequence input/output
from Bio import SeqIO
from Bio.Seq import Seq
#import all sequence records. each gene file is read as one record (Locus to end of Origin)
from Bio.SeqRecord import SeqRecord
#feature location = datatype. categorize and allow parsing through features section
from Bio.SeqFeature import SeqFeature, FeatureLocation

#read record by record (one gene file at a time)
recs = [rec for rec in SeqIO.parse("genbankfiles_justkofirst.gb", "genbank")]


In [2]:
#number of records
len(recs)

20522

In [3]:
#list for records that have target exons
result = []
#list for 
exon_locations = []
#empty list for potential target exons that don't fit within critical region of particular gene
bad_exon = []
count = 0
#list for gene names that don't have 'target exon' in their record (untargeted genes)
untargeted_genes = []

#for loop going through each record 
for rec in recs:
    #separate misc features into datatype. allow parsing of only misc features for start:end values
    feats = [feat for feat in rec.features if feat.type == "misc_feature"]
    #separate exon into datatype. allow parsing of only data under term 'exon'
    exons = [feat for feat in rec.features if feat.type == "exon"]
    # formatting feature key value
    for feat in feats:
      #look for mention of 'critical region' in misc_features datatype. always in 'note' section
      if(feat.qualifiers['note'][0].lower() == "critical region"):
        #print("Start :", feat.location.start) 
        crit_start = feat.location.start
        #print("End :", feat.location.end)
        crit_end = feat.location.end

    crit_exon = []
    for exon in exons:
        
        
        #test for ccdc50
        if (rec.description.split(":")[0].split(" ")[-1] == "Tm4sf4"):
            print(exon.qualifiers)
       
        if("target exon" in exon.qualifiers['note'][0].lower()):   
        #------------------------------------------
            crit_exon.append(exon.qualifiers['note'][0].split(' ')[3])
            if not (exon.location.start > crit_start and exon.location.end < crit_end):
              #print("Exon :", exon)
              bad_exon.append(str(exon.qualifiers['note'][0].split(' ')[3]))
              #target_exon.append(exon.qualifiers['note'][0].split(' ')[3])
              #result.append(exon.qualifiers['note'][0].split(' ')[3])

#logging records that don't have target exon.
    if (len(crit_exon) == 0):
        #search through 'description' in the file and split by commas. append gene name (4th split) into new file 
        untargeted_genes.append(rec.description.split(",")[2].split(" ")[4])
        continue
        
    #checking for multiple target exons within 1 gene. taking only first and last exon ID 
    elif (len(crit_exon) > 1):
      result.append(crit_exon[0])
      exon_locations.append([crit_exon[0],crit_exon[-1]])
      count += 2
    #if gene has only 1 'target exon', append the ensembl ID 
    else:
      result.append(crit_exon[0])
      exon_locations.append(crit_exon[0])
      count += 1
#count is how many mentions of 'target exon' there are. takes into consideration genes that have multiple target exons

#print("total mention of 'target exons':", count)
print("targeted genes: ", len(exon_locations))
print("untargeted genes: ", len(untargeted_genes))



targeted genes:  20436
untargeted genes:  86


In [4]:
#check
if 'ENSMUSE00000155698' in result:
    print('1')

In [5]:
#Writing all critical exon IDs into a file
file = open("crit_exon.txt","w")
for i in range(len(exon_locations)):
  file.write("{} = {}\n".format(result[i],exon_locations[i]))
file.close()

#Writing critical exon IDs that don't fit in critical range into a file
file = open("crit_exon_range.txt","w")
for i in bad_exon:
    file.write(i+"\n")
file.close()

#Writing untargeted gene names into a file 
file = open("untargeted.txt","w")
for i in untargeted_genes:
    file.write(i+"\n")
file.close()

In [6]:
#read in BioMart file containing canonical data on all protein coding genes. 
exons = open('pcg_canonical.txt','r').readlines()
#dictionary for gene IDs, gene names, ranks, trans IDs
gene_ids = {}
#trans_ids = {}
gene_names = {}
rank = {}
trans_ids_exon_first = {}
from collections import defaultdict
trans_ids = defaultdict(dict)

exon_dict = {}
for i in range(1,len(exons)):
  temp = exons[i].split(',')
  exon_dict.setdefault(temp[2], [])
  trans_ids.setdefault(temp[2], {})
  exon_dict[temp[2]] = (temp[1],int(temp[5]),int(temp[6]))
  gene_ids[temp[2]] = temp[0]
  trans_ids[temp[1]][temp[7]] = temp[2]
  trans_ids_exon_first[temp[2]] = temp[1]
  gene_names[temp[2]] = temp[8]
  rank[temp[2]] = temp[7]


In [7]:
print(trans_ids_exon_first['ENSMUSE00000155698'])
trans_ids['ENSMUST00000027263'][max(trans_ids['ENSMUST00000027263'].keys())]

ENSMUST00000027263


'ENSMUSE00000496813'

In [8]:
#list for critical exon IDs from IMPC file that are not present in BioMart pcg_canonical file
non_canonical_exons = []

#fixing ordering of ids. using pcg_canonical file to order the genes with multiple critical exons in correct order (based on rank)
#need correct order to get right start and end phases 
for i,exonid in enumerate(result):
    try:
        if (type(exon_locations[i]) == list):
            if (rank[exon_locations[i][0]] > rank[exon_locations[i][1]]):
                exon_locations[i][0], exon_locations[i][1] = exon_locations[i][1], exon_locations[i][0]
                result[i] = exon_locations[i][0]
                
    #append exon IDs that are not present in pcg_canonical into a new file           
    except KeyError:
        non_canonical_exons.append(exonid)
        
print("#of targeted exons not present in BioMart caonnical file: ", len(non_canonical_exons))
    

#of targeted exons not present in BioMart caonnical file:  607


In [9]:
#re-open file and overwrite previous data. now will have exon IDs in correct order
file = open("crit_exon.txt","w")
for i in range(len(exon_locations)):
  file.write("{} = {}\n".format(result[i],exon_locations[i]))
file.close()

#write non_canonical_exon IDs into a file
file = open("non_canonical_exons.txt","w")
for i in range(len(non_canonical_exons)):
  file.write("{}\n".format(non_canonical_exons[i]))
file.close()

In [10]:
file = open("final.txt","w")
 
#write file including target genes, transript IDs, exon IDs, phases, and rank
#looking for genes with single critical exon, and with multiple
for i in exon_locations:
    try:
        if(type(i) == list):
            file.write("{} = {},{},{},{},{},{}\n".format(gene_names[i[0]],trans_ids_exon_first[i[0]],i[0],i[1],exon_dict[i[0]][1],exon_dict[i[1]][2],rank[i[0]]))
        else:
            #format gene name = transcript ID, crit exon ID, start phase, end phase, rank
            file.write("{} = {},{},{},{},{}\n".format(gene_names[i],trans_ids_exon_first[i],i,exon_dict[i][1],exon_dict[i][2],rank[i]))
    except KeyError:
        continue

file.close()

In [23]:
#list to store start and end phases for each gene
gene_phases = {}
no_coding = [] # FORMAT = [ exonid, geneid, gene name]
print(len(exon_locations))
#CHECK if exon ID exists or not in pcg canonical file
def check(num):
    if (num not in exon_dict.keys()):
        no_coding.append(num)
        return False
    else:
        return True

20436


In [12]:
#lists for different categories 
stop_codon_en2 = [] #phase start 2
read_through = [] #phase 0-1 OR 1-2
frameshift = [] #phase 0-0 OR 0-2 OR 1-0 OR 1-1 
truncated_protein = [] #phase start 2, ends in  -1
no_stop_codon = [] #phase start -
no_proper_end =[]#end with -
unaccounted = []
print(len(gene_phases.keys()))

for i in gene_phases.keys():
    if (gene_phases[i][0] == 2):
        stop_codon_en2.append(i)
    elif (gene_phases[i][0] < 0):
        no_stop_codon.append(i)
    elif (gene_phases[i][1] < 0):
        no_proper_end.append(i)
    elif ((gene_phases[i][0] == 0 and gene_phases[i][1] == 1)):
        read_through.append(i)
    elif ((gene_phases[i][0] == 1 and gene_phases[i][1] == 2)):
        read_through.append(i)
    elif ((gene_phases[i][0] == 0 and gene_phases[i][1] == 2)):
        frameshift.append(i)
    elif ((gene_phases[i][0] == 1 and gene_phases[i][1] == 1)):
        frameshift.append(i)
    elif ((gene_phases[i][0] == 0 and gene_phases[i][1] == 0)):
        frameshift.append(i)
    elif ((gene_phases[i][0] == 1 and gene_phases[i][1] == 0)):
        frameshift.append(i)
    if ((gene_phases[i][0] == 2 and gene_phases[i][1] == -1)):
        truncated_protein.append(i)
    

print("Stop_codon_en2:",len(stop_codon_en2))
print("Readthrough:",len(read_through))
print("Frameshift:",len(frameshift))
print("Unaccounted:",len(unaccounted))

14221
Stop_codon_en2: 2876
Readthrough: 3811
Frameshift: 5108
Unaccounted: 0


In [13]:
stop_codon_file = open('stop_codon.txt','w')
for i in stop_codon_en2:
    stop_codon_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
stop_codon_file.close()

read_through_file = open('read_through.txt','w')
for i in read_through:
    read_through_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
read_through_file.close()

frameshift_file = open('frameshift.txt','w')
for i in frameshift:
    frameshift_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
frameshift_file.close()

truncated_protein_file = open('truncated_protein.txt','w')
for i in truncated_protein:
    truncated_protein_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
truncated_protein_file.close()

no_stop_codon_file = open('no_stop_codon.txt','w')
for i in no_stop_codon:
    no_stop_codon_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
no_stop_codon_file.close()

no_proper_end_file = open('no_proper_end.txt','w')
for i in no_proper_end:
    no_proper_end_file.write("{} = {},{}\n".format(i,gene_ids[i],gene_names[i]))
no_proper_end_file.close()


In [14]:
#Parsing .rpt file as csv 
import csv

filename = "MGI_Gene_Model_Coord.rpt"


gene_model = {}



with open(filename, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    header = next(reader)  # extract the header row
    for row in reader:
        # extract the data from each row
        gene_model[row[10]] = (row[2],row[0].split(":")[1])
#         accession_id.append(row[0].split(":")[1])
#         marker_symbol.append(row[2])
#         ensembl_gene_id.append(row[10])

file = open('mgi_match.txt', 'w')
#6 missing
missing = []
for i in gene_phases.keys():
    try:
        file.write("{} = {},{}\n".format(gene_model[gene_ids[i]][0],gene_ids[i],gene_model[gene_ids[i]][1]))
    except KeyError:
        missing.append(i)

file.close()

In [15]:
import csv

gene_dict = {}

#parse IMPC phenotype per gene file. create dictionary that accounts for Ensembl ID and MGI ID 
with open('phenotypeHitsPerGene.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        row = list(row.values())
        geneid = row[0].split(",")[1].split(":")[1].strip("\"")
        gene_symbol = row[0].split(",")[0]
        phenotype_hits = row[0].split(",")[2].strip("\"")
        desc = row[0].split(",")[3].strip("\"")
        gene_dict[geneid] = (gene_symbol, phenotype_hits, desc)


In [16]:
#example
list(gene_dict.values())[31]

('Psmf1', '1', 'preweaning lethality')

In [17]:
#format(gene_ids[i],gene_dict[gene_model[gene_ids[i]][1]][1]))
#gene_dict[MGIID] = (name,hits)
#gene_model[gene_id] = (name,MGIID)
#gene_ids[exon_id] = gene_id


#go through 3 main categories

#stop codon
file = open('stop_codon_phen.txt','w')
#list to store gene IDs that are within the IMPC targeted mutation file, but not tested for phenotypes 
stop_codon_untested = []
#to count how many genes have 0 phenotypes (tested, but 0)
count = 0
#sum of phenotypes
stop_codon_sum = 0
#sum of preweaning lethalities
lethality_count = 0
for i in stop_codon_en2:
    try:
        if (int(gene_dict[gene_model[gene_ids[i]][1]][1]) == 0):
            count+=1
        #write into a file gene name and no. of phenotypes 
        file.write("{} = {}\n".format(gene_ids[i],gene_dict[gene_model[gene_ids[i]][1]][1]))
        stop_codon_sum += int(gene_dict[gene_model[gene_ids[i]][1]][1])
        if ("preweaning lethality" in gene_dict[gene_model[gene_ids[i]][1]][2]):
            lethality_count += 1
    #write in the gene names that are not tested into list
    except KeyError:
        stop_codon_untested.append((gene_ids[i]))
        
file.close()
print("Stop Codon: 0 phenotypes: ", count)
print("lethal: ",lethality_count)

#Readthrough
file = open('read_through_phen.txt','w')
read_through_untested = []
count = 0
read_through_sum = 0
lethality_count = 0
for i in read_through:
    try:
        if (int(gene_dict[gene_model[gene_ids[i]][1]][1]) == 0):
            count+=1
        file.write("{} = {}\n".format(gene_ids[i],gene_dict[gene_model[gene_ids[i]][1]][1]))
        read_through_sum += int(gene_dict[gene_model[gene_ids[i]][1]][1])
        if ("preweaning lethality" in gene_dict[gene_model[gene_ids[i]][1]][2]):
            lethality_count += 1
    except KeyError:
        read_through_untested.append((gene_ids[i]))
        
file.close()
print("Readthrough: 0 phenotypes: ",count)
print("lethal: ",lethality_count)

#Frameshift
file = open('frameshift_phen.txt','w')
frameshift_untested = []
count = 0
frameshift_sum = 0

lethality_count = 0
for i in frameshift:
    try:
        if (int(gene_dict[gene_model[gene_ids[i]][1]][1]) == 0):
            count+=1
        file.write("{} = {}\n".format(gene_ids[i],gene_dict[gene_model[gene_ids[i]][1]][1]))
        frameshift_sum += int(gene_dict[gene_model[gene_ids[i]][1]][1])
        if ("preweaning lethality" in gene_dict[gene_model[gene_ids[i]][1]][2]):
            lethality_count += 1
    except KeyError:
        frameshift_untested.append((gene_ids[i]))
        
file.close()
print("Frameshift: 0 phenotypes: ",count)
print("lethal: ",lethality_count)

Stop Codon: 0 phenotypes:  258
lethal:  509
Readthrough: 0 phenotypes:  332
lethal:  677
Frameshift: 0 phenotypes:  431
lethal:  877


In [18]:
#Untested
print("Stop_Codon Untested",len(stop_codon_untested))
print("Readthrough Untested: ",len(read_through_untested))
print("Frameshift Untested: ",len(frameshift_untested))
#Sum
print("Stop_Codon Phenotype Sum ",stop_codon_sum)
print("Readthrough Phenotype Sum: ",read_through_sum)
print("Frameshift Phenotype Sum: ",frameshift_sum)

Stop_Codon Untested 1418
Readthrough Untested:  1920
Frameshift Untested:  2552
Stop_Codon Phenotype Sum  6282
Readthrough Phenotype Sum:  8254
Frameshift Phenotype Sum:  11637


In [19]:
import pandas as pd

# read the Excel file
df = pd.read_excel('tm1b.xlsx')

In [20]:
mgi_to_phenotypes = {}
# iterate over the rows and assign the values to variables
for index, row in df.iterrows():
    mgi_id = row[0].split(':')[1]
    phenotypes = row[22]
    # assign the values to other variables as needed
    mgi_to_phenotypes[mgi_id] = phenotypes

In [21]:
frameshift_unique_pheno = []
err = 0
for i in frameshift:
    try:
        frameshift_unique_pheno.append(mgi_to_phenotypes[gene_model[gene_ids[i]][1]])
    except:
        err +=1 

#err = mgids not in xlsx file
print(err)
#getting rid of duplicates
frameshift_unique_pheno = list(dict.fromkeys(frameshift_unique_pheno))
print("Frameshift unique: ",len(frameshift_unique_pheno))

read_through_unique_pheno = []
err = 0
for i in read_through:
    try:
        read_through_unique_pheno.append(mgi_to_phenotypes[gene_model[gene_ids[i]][1]])
    except:
        err +=1 

#err = mgids not in xlsx file
print(err)
#getting rid of duplicates
read_through_unique_pheno = list(dict.fromkeys(read_through_unique_pheno))
print("Read Through unique: ",len(read_through_unique_pheno))


stop_codon_en2_unique_pheno = []
err = 0
for i in stop_codon_en2:
    try:
        stop_codon_en2_unique_pheno.append(mgi_to_phenotypes[gene_model[gene_ids[i]][1]])
    except:
        err +=1 

#err = mgids not in xlsx file
print(err)
#getting rid of duplicates
stop_codon_en2_unique_pheno = list(dict.fromkeys(stop_codon_en2_unique_pheno))
print("Stop Codon unique: ",len(stop_codon_en2_unique_pheno))


3426
Frameshift unique:  343
2550
Read Through unique:  289
1919
Stop Codon unique:  274
